In [ ]:
#SPIN OPERATORS
import numpy as np
from matplotlib import pyplot as plt

#Pauli matrices
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])
I=np.array([[1,0],[0,1]])

#Spin 1/2 matrices (h=1)
S_X=1/2*X
S_Y=1/2*Y
S_Z=1/2*Z
S_element=[S_X,S_Y,S_Z]

#Commutator
def comm(x,y):
    return x@y-y@x # @ is used to multiply matrices

#CLOSED SPIN CHAIN WITH L ELEMNTS (DIMENSION 2^L) optimized version 
def spin_operator(site, op, L): #site is the position of the tensor product where the operator goes. op is one of the Pauli matrices. L is the number of spins
    operators = [I] * L  # operators is a vector of L operators, initially all identities
    operators[site] = op  # We put the Pauli matrix (op) in the place we want [site]. For example, for site=2, op=S_Y ,L=4 we get operators=[I,I,S_y,I]. Now, we need to do the tensor product. 
    result=operators[0]
    for i in range(1,L):
        result=np.kron(result,operators[i]) #for i=0 we are doing operators[0]\otimes operators[1] and we save this in result. In the next step we do result \otimes operators[2] and so on. 
    return result #In our example, spin_operator(2,S_y,4)=I\otimes I\otimes S_Y\otimes Y


#SPIN MATRICES OF EACH PARTICLE IN HILBERT SPACE DIM=2^L=16
L=4 #From now on we work with a 4 spin chain but you only need to change this parameter to change the size of the chain
S=[] #Array, each row has to be composed of the 3 S operators of each particle in the whole hilbert space
for k in range(L):
    S_i=[] #Each row
    for l in S_element: #S_element has the 3 S operators
        S_i.append(spin_operator(k,l,L))
    S.append(S_i)
#Now S[0][1] is the S_Y operator of the 1st particle S_Y*I*I*I, or S[2][2]=I*I*S_Z*I

#print(comm(S[0][0],S[0][1])-1j*S[0][2]) #Check with the commutator that the operators are well defined

#TOTAL SPIN MATRICES (sum over all particles of each spin operator)
S_total=[]
for i in range(3): #loop over S_x S_y and S_z
    S_total.append(S[0][i]) #First we append the s_i=S_x,s_y,S_z of the first particle 
    for j in range(1,L):
        S_total[i]+=S[j][i] #We sum from the second particle (the first is already there) up to the L-th

#TOTAL S^2 OPERATOR
S2=S_total[0]@S_total[0]+S_total[1]@S_total[1]+S_total[2]@S_total[2]
    

In [21]:
#HAMILTONIAN XXX model

H_XXX=S[0][0]@S[1][0]+S[0][1]@S[1][1]+S[0][2]@S[1][2] #we initialize the hamiltonian with the first term in order to start with a matrix

for i in range(1,L): #Now we run the loop from the second term because the first one is already done
    for j in range(3): #Now we sum to perform the dot product of the i spin operator with the i+1 
        H_XXX+=S[i][j]@S[(i+1)%L][j] #IMPORTANT, how do we impose the periodic boundary condition. With (i+1)%L works. % is the modulo operation, makes the division between i+1 and L and returns the reminder. For all i+1<L (all except for the las one) the reminder is indeed i+1 so nothing changes. But for i+1=L (the one that would give problems because you are out of list) the reminder is 0 so we go back to the first spin!
        

In [22]:
#HAMILTONIAN XXZ model

Delta=3 #anisotropy parameter

H_XXZ=S[0][0]@S[1][0]+S[0][1]@S[1][1]+Delta*S[0][2]@S[1][2] 

for i in range(1,L): 
    for j in range(3):  
        if j<2:
            H_XXZ+=S[i][j]@S[(i+1)%L][j] 
        else:
            H_XXZ+=Delta*S[i][j]@S[(i+1)%L][j] 

In [28]:
# CHECKING SYMMETRIES
null_operator = np.zeros((2**L, 2**L), dtype=complex) #null operator no compare if the commutator gives 0 

def check_symm(hamiltonian, symmetry): #we perform the commutator between the hamiltonian and the symmetry operator
    commutator = comm(hamiltonian, symmetry)
    if np.allclose(commutator, null_operator, atol=1e-10):  #Allclose compare each element of both matrices and return true if they are equal. Numpy can have numerical error and even though the commutator is 0 the output can be slightly different, so we tolarate some error with atol
        print("YES, they commute!")
    else:
        print("NO, they don't commute")

# Time to check the symmetries with our hamiltonians

print("Symmetry [H_XXX, S^2]:", end=" ")
check_symm(H_XXX, S2)

print("Symmetry [H_XXX, S_z]:", end=" ")
check_symm(H_XXX, S_total[2])

print("Symmetry [H_XXZ, S^2]:", end=" ")
check_symm(H_XXZ, S2)

print("Symmetry [H_XXZ, S_z]:", end=" ")
check_symm(H_XXZ, S_total[2])


Symmetry [H_XXX, S^2]: YES, they commute!
Symmetry [H_XXX, S_z]: YES, they commute!
Symmetry [H_XXZ, S^2]: NO, they don't commute
Symmetry [H_XXZ, S_z]: YES, they commute!
